In [43]:
import numpy as np

def first_x_sec(seconds, filepath, timecol):
    ''' param: seconds, int, filter out the first seconds of the game
        param: filepath, string, the filepath of the csv datafile
        param: timecol, int, the index of the column that contains time
    '''
    f = open(filepath)
    lines = f.readlines()
    lines = [ line.split(',') for line in lines ]
    if seconds is None:
        return np.array(lines[1:])
    else:
        return np.array(list(filter(lambda line: int(line[timecol])<=seconds, lines[1:])))

# First 5 min: 5*60=300, col 2 is time first line is header
#ability_upgrade = first_x_sec(300, 'dataset/ability_upgrades.csv', 2)
#objectives = first_x_sec(300, 'dataset/objectives.csv', 7)
player_time_data = first_x_sec(300, 'dataset/player_time.csv', 1)

## Match file contains more than just the label
# column 10 contains the True/False value of Radiant win
matches_data = first_x_sec(None, 'dataset/match.csv', None)



In [46]:
labels = np.array([int(match[9] == 'True') for match in matches_data])
print('Shape of player time data matrix')
print(player_time_data.shape)
player_time_data = player_time_data.astype(np.int)

## Separate data matches. Each index in data is one match. One match is a 32*x.
## x is the number of logs. We have data every 60 secods, so for 5 min x=5
games = [np.empty(shape=(32,0), dtype=np.int)]*50000
for x in player_time_data:
    games[x[0]] = np.concatenate((np.array(games[x[0]]), x.reshape(32,1)), axis=1)
#Print the first match log
print("First game data")
print(games[0])

Shape of player time data matrix
(299994, 32)
[[   0    0    0    0    0    0]
 [   0   60  120  180  240  300]
 [   0  409  546  683  956 1056]
 [   0    0    0    1    1    1]
 [   0   63  283  314  485  649]
 [   0  142  622  927 1264 1451]
 [   0    1    4    9   11   13]
 [   0  186  645 1202 1583 1810]
 [   0  168  330  430  530  630]
 [   0    0    0    0    0    0]
 [   0  125  376  376  391  504]
 [   0  200  345  644  919 1102]
 [   0    0    1    6   11   15]
 [   0  193  698 1172 1610 1888]
 [   0  194  628  806 1281 1708]
 [   0    1    5    7   10   17]
 [   0  125  374  570 1216 1633]
 [   0  174  354  614 1082 1300]
 [   0    2    4    8    8   11]
 [   0   77  437  829 1318 1901]
 [   0  138  673  895 1087 1233]
 [   0    1    5    8   10   11]
 [   0   62  543  842 1048 1352]
 [   0  345  684  958 1500 1841]
 [   0    6   12   16   26   32]
 [   0  351  805 1135 1842 2162]
 [   0  100  200  300  400  500]
 [   0    0    0    0    0    0]
 [   0   77  210  210  210  24

# Normalize Data

In [47]:
from sklearn.preprocessing import normalize
for index, game in enumerate(games):
    games[index] = np.vstack((game[0], normalize(game[1:, :])))

/usr/local/lib/python3.6/site-packages/sklearn/utils/validation.py:429: DataConversionWarning: Data with input dtype int64 was converted to float64 by the normalize function.
  warnings.warn(msg, _DataConversionWarning)


In [48]:
from keras.utils.np_utils import to_categorical
labels_binary = to_categorical(labels)

# We don't know how long the game existed for, therefore we append labels_binary for each frame.
data = []
labels_binary_length_corrected = []
for index, game in enumerate(games):
    for frame in game.T:
        data.append(frame[1:])
        labels_binary_length_corrected.append(labels_binary[index])


data = np.array(data)
labels_binary_length_corrected = np.array(labels_binary_length_corrected)

num_train_examples = int(len(data) * 0.9)

x_train = data[:num_train_examples,:]
y_train = labels_binary_length_corrected[:num_train_examples,:]
x_test = data[num_train_examples:,:]
y_test = labels_binary_length_corrected[num_train_examples:,:]

x_train = np.reshape(x_train, x_train.shape + (1,))
x_test = np.reshape(x_test, x_test.shape + (1,))

print(x_train.shape)

Using TensorFlow backend.


(269994, 31, 1)


In [11]:
from keras.layers import Dense, Activation
from keras.models import Sequential
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
import keras

model = Sequential()

model.add(LSTM(
    input_shape=x_train.shape[1:],
    units=50,
    return_sequences=True))
model.add(Dropout(0.2))

model.add(LSTM(
    units=100,
    return_sequences=False))

model.add(Dense(
    units=2))
model.add(Activation('linear'))
#model.compile(loss='mse', optimizer='rmsprop')
model.compile(loss='mse', optimizer='rmsprop', metrics=[keras.metrics.mae, keras.metrics.categorical_accuracy])
#model.compile(loss=keras.losses.categorical_crossentropy,
#             optimizer=keras.optimizers.SGD(lr=0.0001, momentum=0.0, nesterov=False),
#             metrics=[keras.metrics.mae, keras.metrics.categorical_accuracy])

In [12]:
res = model.fit(x_train, y_train, epochs=50, batch_size=1000, validation_split=0.2)

Train on 215995 samples, validate on 53999 samples
Epoch 1/50
215995/215995 [==============================] - 22s - loss: 0.2559 - mean_absolute_error: 0.4994 - categorical_accuracy: 0.5070 - val_loss: 0.2501 - val_mean_absolute_error: 0.4973 - val_categorical_accuracy: 0.5227

In [13]:
# Test loss and accuracy
loss_and_metrics = model.evaluate(x_test, y_test)
print(loss_and_metrics)

30000/30000 [==============================] - 21s     